In [76]:
import cv2
import math
import numpy as np
from PIL import Image
import io

import torch
import torchvision
import torchvision.datasets as datasets
import math 
from matplotlib import pyplot as plt

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# Image Read

In [77]:
dataset_path = './Dataset/MTCNNAlignedCroppedFaces/'

pose_path = './Dataset/inputvector/headpose/'

gaze_path = './Dataset/inputvector/eyegaze/'

blur_path = './Dataset/inputvector/blur/'

position_path = './Dataset/inputvector/faceposition/'

input_path = './Dataset/inputvector/input/'

In [78]:
image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x))
                    for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [79]:
class_names

['private', 'public', 'semiprivate']

In [80]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']
#test_dataset = image_datasets['test']

pri_train = sorted(os.listdir(dataset_path + 'train/private/'))
pub_train = sorted(os.listdir(dataset_path + 'train/public/'))
semi_train = sorted(os.listdir(dataset_path + 'train/semiprivate/'))

pri_val = sorted(os.listdir(dataset_path + 'val/private/'))
pub_val = sorted(os.listdir(dataset_path + 'val/public/'))
semi_val = sorted(os.listdir(dataset_path + 'val/semiprivate/'))

#pri_test = sorted(os.listdir(dataset_path + 'test/private/'))
#pub_test = sorted(os.listdir(dataset_path + 'test/public/'))
#semi_test = sorted(os.listdir(dataset_path + 'test/semiprivate/'))

In [81]:
pri_train_filelist = [x.split('.')[0] for x in pri_train]
pub_train_filelist = [x.split('.')[0] for x in pub_train]
semi_train_filelist = [x.split('.')[0] for x in semi_train]

pri_val_filelist = [x.split('.')[0] for x in pri_val]
pub_val_filelist = [x.split('.')[0] for x in pub_val]
semi_val_filelist = [x.split('.')[0] for x in semi_val]

#pri_test_filelist = [x.split('.')[0] for x in pri_test]
#pub_test_filelist = [x.split('.')[0] for x in pub_test]
#semi_test_filelist = [x.split('.')[0] for x in semi_test]

In [82]:
print(pri_train_filelist[:5])
print(pub_train_filelist[:5])
print(semi_train_filelist[:5])

print(pri_val_filelist[:5])
print(pub_val_filelist[:5])
print(semi_val_filelist[:5])

#print(pri_test_filelist[:5])
#print(pub_test_filelist[:5])
#print(semi_test_filelist[:5])

['00000ae5e7fcc87222f1fb6f221e7501558e5b08_0', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_1', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_2', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_3', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_4']
['00bc0b7df6951edbe08ac69cfb00f0b7a9927a31_0', '00bc0b7df6951edbe08ac69cfb00f0b7a9927a31_1', '00bc0b7df6951edbe08ac69cfb00f0b7a9927a31_2', '00bde3f3037497aaa1e665ba1b23fc149ec9081f_0', '00c5d8c5b83e61cfcf1bd0dffb0911230d25ce7b_0']
['00ce1cf572bcf97c138129087291ba7cce7ed8b3_0', '00ce1cf572bcf97c138129087291ba7cce7ed8b3_1', '00ce1cf572bcf97c138129087291ba7cce7ed8b3_2', '00ce1cf572bcf97c138129087291ba7cce7ed8b3_3', '0339520eef09650d55879247cba49c752a2c0b1e_0']
['04085231fc015457761e587ee53e4d1be6eb3666_0', '04085231fc015457761e587ee53e4d1be6eb3666_1', '04085231fc015457761e587ee53e4d1be6eb3666_2', '04085231fc015457761e587ee53e4d1be6eb3666_3', '11708847396_9383898125_b_0']
['052e744bb87e87adebaf05d4da1fea43c91519be_0', '052e744bb87e87adebaf05d4da1fea43c91519be_1',

In [83]:
train_filelist = pri_train_filelist  + pub_train_filelist + semi_train_filelist
val_filelist = pri_val_filelist  + pub_val_filelist + semi_val_filelist
#test_filelist = pri_test_filelist  + pub_test_filelist + semi_test_filelist

In [84]:
print(len(training_dataset))
print(len(validation_dataset))
#print(len(test_dataset))

6598
1195


# Head Pose Estimation

In [85]:
def face_orientation(bounding_boxs, landmarks):
    size = (bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0]) #(height, width, color_channel) 

    image_points = np.array([
                            (landmarks[2], landmarks[7]),     # Nose tip
                            (landmarks[1], landmarks[6]),     # Left eye left corner
                            (landmarks[0], landmarks[5]),     # Right eye right corne
                            (landmarks[3], landmarks[8]),     # Left Mouth corner
                            (landmarks[4], landmarks[9])     # Right mouth corner
                          #  (landmarks[10], landmarks[11]) 
                        ], dtype="double")

    

                        
    model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (-165.0, 170.0, -135.0),     # Left eye left corner
                            (165.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                          #  (0.0, -330.0, -65.0)
                        ])
    
    # Camera internals
 
    center = (size[1]/2, size[0]/2)
    focal_length = center[0] / np.tan(60/2 * np.pi / 180)
    camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.cv2.SOLVEPNP_ITERATIVE)

    
    axis = np.float32([[500,0,0], 
                          [0,500,0], 
                          [0,0,500]])
                          
    imgpts, jac = cv2.projectPoints(axis, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    modelpts, jac2 = cv2.projectPoints(model_points, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    rvec_matrix = cv2.Rodrigues(rotation_vector)[0]

    proj_matrix = np.hstack((rvec_matrix, translation_vector))
    eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6] 

    
    pitch, yaw, roll = [math.radians(_) for _ in eulerAngles]


    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))


    return (int(roll), int(pitch), int(yaw)), (landmarks[4], landmarks[5])


In [6]:
def face_orientation(bounding_boxes, landmarks):
    
    size = (bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0]) #(height, width, color_channel)

    image_points = np.array([
                            (landmarks[2], landmarks[7]),     # Nose tip
                            (landmarks[1], landmarks[6]),     # Left eye left corner
                            (landmarks[0], landmarks[5]),     # Right eye right corne
                            (landmarks[3], landmarks[8]),     # Left Mouth corner
                            (landmarks[4], landmarks[9])      # Right mouth corner
                        ], dtype="double")

    

                        
    model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (-165.0, 170.0, -135.0),     # Left eye left corner
                            (165.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner                         
                        ])
    
    # Camera internals
 
    center = (size[1]/2, size[0]/2)
    focal_length = center[0] / np.tan(60/2 * np.pi / 180)
    camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.cv2.SOLVEPNP_ITERATIVE)

    
    # calculate rotation angles
    theta = cv2.norm(rotation_vector, cv2.NORM_L2)
    
    # transformed to quaterniond
    w = math.cos(theta / 2)
    x = math.sin(theta / 2)*rotation_vector[0][0] / theta
    y = math.sin(theta / 2)*rotation_vector[1][0] / theta
    z = math.sin(theta / 2)*rotation_vector[2][0] / theta

    ysqr = y * y
    # pitch (x-axis rotation)
    t0 = 2.0 * (w * x + y * z)
    t1 = 1.0 - 2.0 * (x * x + ysqr)
    print('t0:{}, t1:{}'.format(t0, t1))
    pitch = math.atan2(t0, t1) - 0.8356857

    # yaw (y-axis rotation)
    t2 = 2.0 * (w * y - z * x)
    if t2 > 1.0:
        t2 = 1.0
    if t2 < -1.0:
        t2 = -1.0
    yaw = math.asin(t2) + 0.005409

    # roll (z-axis rotation)
    t3 = 2.0 * (w * z + x * y)
    t4 = 1.0 - 2.0 * (ysqr + z * z)
    roll = math.atan2(t3, t4) - 2.573345436
    
    return (int(roll), int(pitch), int(yaw)), (landmarks[4], landmarks[5])


In [68]:
face_coor = np.load('./Dataset/FaceCoordinates/train/public/5673334536_db313918c6_b.npz')
bounding_boxes = face_coor['a'][0]
landmarks = face_coor['b'][0]
rotate_degree, nose = face_orientation(bounding_boxes, landmarks)
rotate_degree

for i in range(len(training_dataset)):
    image, label = training_dataset[i]
    print(train_filelist[i])
    print(i)
    try:
        if label == 0:
            if os.path.isfile(pose_path + 'train/private/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/private/' + train_filelist[i] + '.jpg'
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/train/private/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            landmarks = coordinate['b'][face_num] 
            rotate_degree, nose = face_orientation(bounding_boxes, landmarks)
            # save the head pose angle            
            np.save(pose_path + 'train/private/' + train_filelist[i] + '.npy' , rotate_degree)

        elif label == 1:
            if os.path.isfile(pose_path + 'train/public/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/public/' + train_filelist[i] + '.jpg'
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/train/public/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            landmarks = coordinate['b'][face_num] 
            rotate_degree, nose = face_orientation(bounding_boxes, landmarks)
            # save the head pose angle            
            np.save(pose_path + 'train/public/' + train_filelist[i] + '.npy' , rotate_degree)
        else:
            if os.path.isfile(pose_path + 'train/semiprivate/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/semiprivate/' + train_filelist[i] + '.jpg'
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/train/semiprivate/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            landmarks = coordinate['b'][face_num] 
            rotate_degree, nose = face_orientation(bounding_boxes, landmarks)
            # save the head pose angle            
            np.save(pose_path + 'train/semiprivate/' + train_filelist[i] + '.npy' , rotate_degree)
    except ValueError:
        print('No faces detected for ' + train_filelist[i] + ". Also Dlib failed.")
        if label == 0:
            np.save(pose_dataset_path + 'train/private/' + train_filelist[i] , rotate_degree=np.zeros(1))
        elif label == 1:
            np.save(pose_dataset_path + 'train/public/' + train_filelist[i] , rotate_degree=np.zeros(1))
        else:
            np.save(pose_dataset_path + 'train/semiprivate/' + train_filelist[i] , rotate_degree=np.zeros(1))
        continue

t0:0.06074357152099258, t1:-0.9513717401769182


(-2, 2, 0)

# Bluriness

In [86]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [12]:
for i in range(len(training_dataset)):
    image, label = training_dataset[i]
    print(train_filelist[i])
    print(i)
    try:
        if label == 0:
            if os.path.isfile(blur_path + 'train/private/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/private/' + train_filelist[i] + '.jpg'
            face_image = cv2.imread(face_file)
            face_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
            
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            image = cv2.imread('./Dataset/images/train/private/' + image_name + '.jpg')
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            fm_ratio = variance_of_laplacian(face_gray)/variance_of_laplacian(image_gray)
            # save the head pose angle            
            np.save(blur_path + 'train/private/' + train_filelist[i] + '.npy' , fm_ratio)

        elif label == 1:
            if os.path.isfile(blur_path + 'train/public/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/public/' + train_filelist[i] + '.jpg'
            face_image = cv2.imread(face_file)
            face_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
            
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            image = cv2.imread('./Dataset/images/train/public/' + image_name + '.jpg')
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            fm_ratio = variance_of_laplacian(face_gray)/variance_of_laplacian(image_gray)
            # save the head pose angle            
            np.save(blur_path + 'train/public/' + train_filelist[i] + '.npy' , fm_ratio)
        else:
            if os.path.isfile(blur_path + 'train/semiprivate/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/semiprivate/' + train_filelist[i] + '.jpg'
            face_image = cv2.imread(face_file)
            face_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
            
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            image = cv2.imread('./Dataset/images/train/semiprivate/' + image_name + '.jpg')
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            fm_ratio = variance_of_laplacian(face_gray)/variance_of_laplacian(image_gray)
            # save the head pose angle            
            np.save(blur_path + 'train/semiprivate/' + train_filelist[i] + '.npy' , fm_ratio)
    except ValueError:
        print('No faces detected for ' + train_filelist[i] + ". Also Dlib failed.")
        if label == 0:
            np.save(blur_dataset_path + 'train/private/' + train_filelist[i] , fm_ratio=np.zeros(1))
        elif label == 1:
            np.save(blur_dataset_path + 'train/public/' + train_filelist[i] , fm_ratio=np.zeros(1))
        else:
            np.save(blur_dataset_path + 'train/semiprivate/' + train_filelist[i] , fm_ratio=np.zeros(1))
        continue

00000ae5e7fcc87222f1fb6f221e7501558e5b08_0
0
00000ae5e7fcc87222f1fb6f221e7501558e5b08_1
1
00000ae5e7fcc87222f1fb6f221e7501558e5b08_2
2
00000ae5e7fcc87222f1fb6f221e7501558e5b08_3
3
00000ae5e7fcc87222f1fb6f221e7501558e5b08_4
4
00000ae5e7fcc87222f1fb6f221e7501558e5b08_5
5
00000ae5e7fcc87222f1fb6f221e7501558e5b08_6
6
00000ae5e7fcc87222f1fb6f221e7501558e5b08_7
7
00ae730b3a1b68358d4bda73978b14707cd1ef6c_0
8
00ae730b3a1b68358d4bda73978b14707cd1ef6c_1
9
00af5da9ae2bca79dd46100a485fd13bba848972_0
10
00af5da9ae2bca79dd46100a485fd13bba848972_1
11
00b1297d36c7dd6470d9439df065a10b3b241f40_0
12
00b1297d36c7dd6470d9439df065a10b3b241f40_1
13
00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b_0
14
00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b_1
15
00c3b8f6898f99bfa77bf10a1492fb959336c158_0
16
00c3b8f6898f99bfa77bf10a1492fb959336c158_1
17
00d40a80e5cbcc09883ce1ad79e79b16879786e5_0
18
00d40a80e5cbcc09883ce1ad79e79b16879786e5_1
19
00d40a80e5cbcc09883ce1ad79e79b16879786e5_2
20
00d40a80e5cbcc09883ce1ad79e79b16879786e5_3
2

7105c62c8083686f2d86b9acbf13f6e663d93b86_1
352
7105c62c8083686f2d86b9acbf13f6e663d93b86_2
353
7105c62c8083686f2d86b9acbf13f6e663d93b86_3
354
717c1ce425ddf48bb3424243243d753051c78324_0
355
717c1ce425ddf48bb3424243243d753051c78324_1
356
7238881ddb58768da739c538d87dc4ff50ab64f0_0
357
7238881ddb58768da739c538d87dc4ff50ab64f0_1
358
7238881ddb58768da739c538d87dc4ff50ab64f0_2
359
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_0
360
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_1
361
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_2
362
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_3
363
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_4
364
73542fb0b835191230401c7ea31d180ab4b28524_0
365
73542fb0b835191230401c7ea31d180ab4b28524_1
366
73542fb0b835191230401c7ea31d180ab4b28524_2
367
73542fb0b835191230401c7ea31d180ab4b28524_3
368
7509ba9c6bee9adc0addfea061b88805b3996cfe_0
369
7509ba9c6bee9adc0addfea061b88805b3996cfe_1
370
7509ba9c6bee9adc0addfea061b88805b3996cfe_2
371
7509ba9c6bee9adc0addfea061b88805b3996cfe_3
372
7509ba9c6bee9

af8f5db306215ab0c19ba07625ce4da653bfe5fc_1
711
af8f5db306215ab0c19ba07625ce4da653bfe5fc_2
712
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_0
713
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_1
714
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_2
715
army_base_Google_005_0
716
army_base_Google_005_1
717
army_base_Google_007_0
718
army_base_Google_007_1
719
army_base_Google_020_0
720
army_base_Google_020_1
721
army_base_Google_021_0
722
army_base_Google_021_1
723
army_base_Google_021_2
724
army_base_Google_021_3
725
army_base_Google_028_0
726
army_base_Google_028_1
727
army_base_Google_028_2
728
army_base_Google_029_0
729
army_base_Google_032_0
730
army_base_Google_032_1
731
army_base_Google_032_2
732
army_base_Google_032_3
733
army_base_Google_045_0
734
army_base_Google_045_1
735
army_base_Google_045_2
736
army_base_Google_045_3
737
army_base_Google_045_4
738
army_base_Google_045_5
739
army_base_Google_045_6
740
army_base_Google_050_0
741
army_base_Google_050_1
742
army_base_Google_062_0
743
army_base

frame_ed1w18dfm79jznds_5
1190
frame_ed1w18dfm79jznds_6
1191
frame_fl8bo8dieg1925bt_0
1192
frame_fl8bo8dieg1925bt_1
1193
frame_fl8bo8dieg1925bt_2
1194
frame_hdepjixs67t3l4zo_0
1195
frame_hdepjixs67t3l4zo_1
1196
frame_hdepjixs67t3l4zo_2
1197
frame_hdepjixs67t3l4zo_3
1198
frame_j4304ccu00q3hcbk_0
1199
frame_j4304ccu00q3hcbk_1
1200
frame_j4304ccu00q3hcbk_2
1201
frame_j4304ccu00q3hcbk_3
1202
frame_j4304ccu00q3hcbk_4
1203
frame_j4304ccu00q3hcbk_5
1204
frame_j4304ccu00q3hcbk_6
1205
frame_kbflmzut9m5suw2k_0
1206
frame_kbflmzut9m5suw2k_1
1207
frame_kbflmzut9m5suw2k_2
1208
frame_kbflmzut9m5suw2k_3
1209
frame_kbflmzut9m5suw2k_4
1210
frame_ke4v6e87c07xb4v6_0
1211
frame_ke4v6e87c07xb4v6_1
1212
frame_ke4v6e87c07xb4v6_2
1213
frame_ke4v6e87c07xb4v6_3
1214
frame_ke4v6e87c07xb4v6_4
1215
frame_ke4v6e87c07xb4v6_5
1216
frame_ke4v6e87c07xb4v6_6
1217
frame_ke4v6e87c07xb4v6_7
1218
frame_ks5bck7yg63vs91o_0
1219
frame_ks5bck7yg63vs91o_1
1220
frame_ks5bck7yg63vs91o_10
1221
frame_ks5bck7yg63vs91o_2
1222
frame_ks5

0ba55793ce093fcd12410b63bae05ab156365484_0
1744
0ba55793ce093fcd12410b63bae05ab156365484_1
1745
0ba55793ce093fcd12410b63bae05ab156365484_2
1746
0ba55793ce093fcd12410b63bae05ab156365484_3
1747
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_0
1748
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_1
1749
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_2
1750
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_3
1751
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_4
1752
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_5
1753
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_6
1754
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_7
1755
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_8
1756
0bb3d7fcd5568d33af9f9c7ba15a1f5a24b9f1f7_9
1757
0c66ac87b1f6e2e6eea47f7abf387067594b2ca6_0
1758
0c66ac87b1f6e2e6eea47f7abf387067594b2ca6_1
1759
0c66ac87b1f6e2e6eea47f7abf387067594b2ca6_2
1760
0c66ac87b1f6e2e6eea47f7abf387067594b2ca6_3
1761
0c66ac87b1f6e2e6eea47f7abf387067594b2ca6_4
1762
0ca17aa504a41789182f1f07f41325fb5011539d_0
1763
0ca17aa504a41789182f1f07f41325fb5011539d

...

6b6fcf1c477fce2be3a70336f2ba191e9103711d_3
4487
6b6fcf1c477fce2be3a70336f2ba191e9103711d_4
4488
6b6fcf1c477fce2be3a70336f2ba191e9103711d_5
4489
6b6fcf1c477fce2be3a70336f2ba191e9103711d_6
4490
6b6fcf1c477fce2be3a70336f2ba191e9103711d_7
4491
6b6fcf1c477fce2be3a70336f2ba191e9103711d_8
4492
6b6fcf1c477fce2be3a70336f2ba191e9103711d_9
4493
6d025d79d938bd06d3abb73453eb467cbf06b81a_0
4494
6d025d79d938bd06d3abb73453eb467cbf06b81a_1
4495
6d025d79d938bd06d3abb73453eb467cbf06b81a_2
4496
6d025d79d938bd06d3abb73453eb467cbf06b81a_3
4497
6d025d79d938bd06d3abb73453eb467cbf06b81a_4
4498
6d025d79d938bd06d3abb73453eb467cbf06b81a_5
4499
6d025d79d938bd06d3abb73453eb467cbf06b81a_6
4500
6d025d79d938bd06d3abb73453eb467cbf06b81a_7
4501
6d025d79d938bd06d3abb73453eb467cbf06b81a_8
4502
6d025d79d938bd06d3abb73453eb467cbf06b81a_9
4503
6d63583f99494f5465d5b2667c68dca781e02d40_0
4504
6d63583f99494f5465d5b2667c68dca781e02d40_1
4505
6dadbe658729996bde3e7275615a279548b7e3e8_0
4506
6dadbe658729996bde3e7275615a279548b7e3e8

COCO_train2014_000000074515_1
4860
COCO_train2014_000000074515_2
4861
COCO_train2014_000000074515_3
4862
COCO_train2014_000000074515_4
4863
Cohort 2_Chem Lab I_ Distillation experiment_0
4864
Cohort 2_Chem Lab I_ Distillation experiment_1
4865
Cohort 2_Chem Lab I_ Distillation experiment_2
4866
Cohort 2_Chem Lab I_ Distillation experiment_3
4867
Cohort 2_Chem Lab I_ Distillation experiment_4
4868
Cohort 2_Chem Lab I_ Distillation experiment_5
4869
Course Catalog_670x465_0
4870
Course Catalog_670x465_1
4871
Course Catalog_670x465_2
4872
Course Catalog_670x465_3
4873
Course Catalog_670x465_4
4874
D943_242_273_1200_0
4875
D943_242_273_1200_1
4876
D943_242_273_1200_2
4877
D943_242_273_1200_3
4878
D984_72_167_1200_0
4879
D984_72_167_1200_1
4880
D984_72_167_1200_2
4881
D984_72_167_1200_3
4882
D984_72_167_1200_4
4883
D984_72_167_1200_5
4884
D984_72_167_1200_6
4885
D984_72_167_1200_7
4886
DSC_4732_0
4887
DSC_4732_1
4888
DSC_4732_2
4889
DSC_4732_3
4890
DSC_4732_4
4891
Durham_hero_pathwaybenefit

classroom_Google_059_0
5303
classroom_Google_060_0
5304
classroom_Google_060_1
5305
classroom_Google_061_0
5306
classroom_Google_061_1
5307
classroom_Google_066_0
5308
classroom_Google_066_1
5309
classroom_Google_066_2
5310
classroom_Google_066_3
5311
classroom_Google_068_0
5312
classroom_Google_068_1
5313
classroom_Google_069_0
5314
classroom_Google_069_1
5315
classroom_Google_069_2
5316
classroom_Google_070_0
5317
classroom_Google_070_1
5318
classroom_Google_071_0
5319
classroom_Google_071_1
5320
classroom_Google_072_0
5321
classroom_Google_072_1
5322
classroom_Google_072_2
5323
classroom_Google_074_0
5324
classroom_Google_074_1
5325
classroom_Google_075_0
5326
classroom_Google_075_1
5327
classroom_Google_077_0
5328
classroom_Google_077_1
5329
classroom_Google_077_2
5330
classroom_Google_077_3
5331
classroom_Google_077_4
5332
classroom_Google_078_0
5333
classroom_Google_078_1
5334
classroom_Google_081_0
5335
classroom_Google_081_1
5336
classroom_Google_084_0
5337
classroom_Google_084

frame_5yjsc955gxc7qisp_1
5779
frame_6gmtv0nc9klfzvn8_0
5780
frame_6gmtv0nc9klfzvn8_1
5781
frame_6gmtv0nc9klfzvn8_10
5782
frame_6gmtv0nc9klfzvn8_11
5783
frame_6gmtv0nc9klfzvn8_12
5784
frame_6gmtv0nc9klfzvn8_2
5785
frame_6gmtv0nc9klfzvn8_3
5786
frame_6gmtv0nc9klfzvn8_4
5787
frame_6gmtv0nc9klfzvn8_5
5788
frame_6gmtv0nc9klfzvn8_6
5789
frame_6gmtv0nc9klfzvn8_7
5790
frame_6gmtv0nc9klfzvn8_8
5791
frame_6gmtv0nc9klfzvn8_9
5792
frame_7scy8m3m8q3qpnnv_0
5793
frame_7scy8m3m8q3qpnnv_1
5794
frame_7scy8m3m8q3qpnnv_2
5795
frame_7scy8m3m8q3qpnnv_3
5796
frame_7scy8m3m8q3qpnnv_4
5797
frame_7scy8m3m8q3qpnnv_5
5798
frame_7wyexeko49kea3y4_0
5799
frame_7wyexeko49kea3y4_1
5800
frame_7wyexeko49kea3y4_2
5801
frame_8o4lhcl4zw216bis_0
5802
frame_8o4lhcl4zw216bis_1
5803
frame_8o4lhcl4zw216bis_2
5804
frame_8o4lhcl4zw216bis_3
5805
frame_8o4lhcl4zw216bis_4
5806
frame_8o4lhcl4zw216bis_5
5807
frame_8o4lhcl4zw216bis_6
5808
frame_92xu023f264wt62w_0
5809
frame_92xu023f264wt62w_1
5810
frame_banhg5nf6zmt6xwb_0
5811
frame_b

lecture_room_Google_432_4
6286
lecture_room_Google_437_0
6287
lecture_room_Google_445_0
6288
lecture_room_Google_459_0
6289
lecture_room_Google_459_1
6290
lecture_room_Google_473_0
6291
lecture_room_Google_473_1
6292
lecture_room_Google_473_2
6293
lecture_room_Google_473_3
6294
lecture_room_Google_473_4
6295
lecture_room_Google_473_5
6296
lecture_room_Google_473_6
6297
lecture_room_Google_473_7
6298
lecture_room_Google_493_0
6299
lecture_room_Google_493_1
6300
lecture_room_Google_493_2
6301
lecture_room_Google_493_3
6302
lecture_room_Google_517_0
6303
lecture_room_Google_517_1
6304
lecture_room_Google_517_2
6305
lecture_room_Google_517_3
6306
lecture_room_Google_540_0
6307
lecture_room_Google_540_1
6308
lecture_room_Google_540_2
6309
lecture_room_Google_540_3
6310
lecture_room_Google_540_4
6311
lecture_room_Google_549_0
6312
lecture_room_Google_549_1
6313
lecture_room_Google_549_2
6314
lecture_room_Google_562_0
6315
lecture_room_Google_562_1
6316
lecture_room_Google_562_2
6317
lecture_

In [25]:
for i in range(len(validation_dataset)):
    image, label = validation_dataset[i]
    print(val_filelist[i])
    print(i)
    try:
        if label == 0:
            if os.path.isfile(blur_path + 'val/private/' + val_filelist[i] + '.npy'):
                print(val_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'val/private/' + val_filelist[i] + '.jpg'
            face_image = cv2.imread(face_file)
            face_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
            
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            image = cv2.imread('./Dataset/images/val/private/' + image_name + '.jpg')
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            fm_ratio = variance_of_laplacian(face_gray)/variance_of_laplacian(image_gray)
            # save the head pose angle            
            np.save(blur_path + 'val/private/' + val_filelist[i] + '.npy' , fm_ratio)

        elif label == 1:
            if os.path.isfile(blur_path + 'val/public/' + val_filelist[i] + '.npy'):
                print(val_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'val/public/' + val_filelist[i] + '.jpg'
            face_image = cv2.imread(face_file)
            face_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
            
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            image = cv2.imread('./Dataset/images/val/public/' + image_name + '.jpg')
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            fm_ratio = variance_of_laplacian(face_gray)/variance_of_laplacian(image_gray)
            # save the head pose angle            
            np.save(blur_path + 'val/public/' + val_filelist[i] + '.npy' , fm_ratio)
        else:
            if os.path.isfile(blur_path + 'val/semiprivate/' + val_filelist[i] + '.npy'):
                print(val_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'val/semiprivate/' + val_filelist[i] + '.jpg'
            face_image = cv2.imread(face_file)
            face_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
            
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            image = cv2.imread('./Dataset/images/val/semiprivate/' + image_name + '.jpg')
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            fm_ratio = variance_of_laplacian(face_gray)/variance_of_laplacian(image_gray)
            # save the head pose angle            
            np.save(blur_path + 'val/semiprivate/' + val_filelist[i] + '.npy' , fm_ratio)
    except ValueError:
        print('No faces detected for ' + val_filelist[i] + ". Also Dlib failed.")
        if label == 0:
            np.save(blur_dataset_path + 'val/private/' + val_filelist[i] , fm_ratio=np.zeros(1))
        elif label == 1:
            np.save(blur_dataset_path + 'val/public/' + val_filelist[i] , fm_ratio=np.zeros(1))
        else:
            np.save(blur_dataset_path + 'val/semiprivate/' + val_filelist[i] , fm_ratio=np.zeros(1))
        continue

04085231fc015457761e587ee53e4d1be6eb3666_0
0
04085231fc015457761e587ee53e4d1be6eb3666_1
1
04085231fc015457761e587ee53e4d1be6eb3666_2
2
04085231fc015457761e587ee53e4d1be6eb3666_3
3
11708847396_9383898125_b_0
4
11708847396_9383898125_b_1
5
12772838113_10d6e384c0_b_0
6
12772838113_10d6e384c0_b_1
7
12772838113_10d6e384c0_b_2
8
12772897004_3573abfe77_b_0
9
12772897004_3573abfe77_b_1
10
12772897004_3573abfe77_b_2
11
12772897004_3573abfe77_b_3
12
12772897004_3573abfe77_b_4
13
137300_0
14
137300_1
15
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_0
16
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_1
17
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_2
18
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_3
19
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_4
20
143eb889d808e38195c2ac1aa20139e285905aa0_0
21
143eb889d808e38195c2ac1aa20139e285905aa0_1
22
143eb889d808e38195c2ac1aa20139e285905aa0_2
23
143eb889d808e38195c2ac1aa20139e285905aa0_3
24
14938114374_89dd0c0566_b_0
25
14938114374_89dd0c0566_b_1
26
14938114374_89dd0c0566_b_2
2

33d21955d18d74715d844b9bae2ce0d0bf5dcfc2_2
411
366577_orig_0
412
366577_orig_1
413
366577_orig_2
414
366577_orig_3
415
366577_orig_4
416
366577_orig_5
417
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_0
418
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_1
419
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_2
420
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_3
421
49dae06853e5650bf3862ffcc5ca12c28f30bbc2_0
422
49dae06853e5650bf3862ffcc5ca12c28f30bbc2_1
423
50a4cddc5bfa2709f962f8eb13459739adc6e2b6_0
424
50a4cddc5bfa2709f962f8eb13459739adc6e2b6_1
425
50e5269163e82037e475842fdb28113894ce2c80_0
426
50e5269163e82037e475842fdb28113894ce2c80_1
427
50e5269163e82037e475842fdb28113894ce2c80_2
428
53a40ea9e0fb33b56ff4088c1a60b03dc4c14cd7_0
429
53a40ea9e0fb3

25216282-asian-student-raising-her-hand-in-a-university-classroom_6
777
25216282-asian-student-raising-her-hand-in-a-university-classroom_7
778
25216282-asian-student-raising-her-hand-in-a-university-classroom_8
779
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_0
780
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_1
781
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_2
782
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_3
783
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_4
784
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_5
785
290e64555c20ba308adb8f7a7867f6b15be31475_0
786
290e64555c20ba308adb8f7a7867f6b15be31475_1
787
29af9a009bfd6501f625ea926bd630bc2047b572_0
788
29af9a009bfd6501f625ea926bd630bc2047b572_1
789
29af9a009bfd6501f625ea926bd630bc2047b572_2
790
29af9a009bfd6501f625ea926bd630bc2047b572_3
791
30505970f0e1b25ab0a3a1459e97505b09f60dd7_0
792
30505970f0e1b25ab0a3a1459e97505b09f60dd7_1
793
30505970f0e1b25ab0a3a145

public_transportation_028_4
1163
public_transportation_081_0
1164
public_transportation_081_1
1165
public_transportation_083_0
1166
public_transportation_083_1
1167
public_transportation_111_0
1168
public_transportation_111_1
1169
public_transportation_111_2
1170
public_transportation_111_3
1171
public_transportation_145_0
1172
public_transportation_145_1
1173
public_transportation_145_2
1174
public_transportation_145_3
1175
public_transportation_207_0
1176
public_transportation_207_1
1177
public_transportation_207_2
1178
public_transportation_207_3
1179
public_transportation_296_0
1180
public_transportation_296_1
1181
public_transportation_296_2
1182
public_transportation_296_3
1183
public_transportation_296_4
1184
public_transportation_296_5
1185
public_transportation_296_6
1186
public_transportation_386_0
1187
public_transportation_386_1
1188
public_transportation_450_0
1189
public_transportation_450_1
1190
public_transportation_450_2
1191
public_transportation_450_3
1192
shoeshop_0

# Face position, Face Size ratio and Distance with Centroids

In [16]:
for i in range(len(training_dataset)):
    image, label = training_dataset[i]
    print(train_filelist[i])
    print(i)
    y_cen = 0
    y_weight = 0
    x_cen = 0
    x_weight = 0
    try:
        if label == 0:            
            if os.path.isfile(position_path + 'train/private/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/private/' + train_filelist[i] + '.jpg'
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/train/private/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            size = np.array([bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0],(bounding_boxes[1]+bounding_boxes[3])/2, (bounding_boxes[0]+bounding_boxes[2])/2])#(up, left, bottom, right)
            image = cv2.imread('./Dataset/images/train/private/' + image_name + '.jpg')
            height, width = image.shape[:2]
            for a in range(len(coordinate['a'])):
                blur = np.load(blur_path + 'train/private/' + train_filelist[i] + '.npy')
                y_cen += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*(coordinate['a'][a-1][1]+coordinate['a'][a-1][3])*blur/2
                y_weight += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*blur/2
                x_cen += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*(coordinate['a'][a-1][2]+coordinate['a'][a-1][0])*blur/2
                x_weight += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*blur/2
            y_cen = y_cen / y_weight
            x_cen = x_cen / x_weight
            dis_cen = (y_cen-size[2])**2 + (x_cen-size[3])**2
            size_ratio = np.array([size[0]/height, size[1]/width, size[2]/height, size[3]/width, dis_cen])
            # save the head pose angle            
            np.save(position_path + 'train/private/' + train_filelist[i] + '.npy' , size_ratio)

        elif label == 1:
            if os.path.isfile(position_path + 'train/public/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/public/' + train_filelist[i] + '.jpg'
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/train/public/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            size = np.array([bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0],(bounding_boxes[1]+bounding_boxes[3])/2, (bounding_boxes[0]+bounding_boxes[2])/2])#(up, left, bottom, right)
            image = cv2.imread('./Dataset/images/train/public/' + image_name + '.jpg')
            height, width = image.shape[:2]
            for a in range(len(coordinate['a'])):
                blur = np.load(blur_path + 'train/public/' + train_filelist[i] + '.npy')
                y_cen += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*(coordinate['a'][a-1][1]+coordinate['a'][a-1][3])*blur/2
                y_weight += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*blur/2
                x_cen += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*(coordinate['a'][a-1][2]+coordinate['a'][a-1][0])*blur/2
                x_weight += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*blur/2
            y_cen = y_cen / y_weight
            x_cen = x_cen / x_weight
            dis_cen = (y_cen-size[2])**2 + (x_cen-size[3])**2
            size_ratio = np.array([size[0]/height, size[1]/width, size[2]/height, size[3]/width, dis_cen])
            # save the head pose angle            
            np.save(position_path + 'train/public/' + train_filelist[i] + '.npy' , size_ratio)
        else:
            if os.path.isfile(position_path + 'train/semiprivate/' + train_filelist[i] + '.npy'):
                print(train_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'train/semiprivate/' + train_filelist[i] + '.jpg'
            face_num = int(train_filelist[i].rsplit('_')[-1])
            image_name = train_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/train/semiprivate/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            size = np.array([bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0],(bounding_boxes[1]+bounding_boxes[3])/2, (bounding_boxes[0]+bounding_boxes[2])/2])#(up, left, bottom, right)
            image = cv2.imread('./Dataset/images/train/semiprivate/' + image_name + '.jpg')
            height, width = image.shape[:2]
            for a in range(len(coordinate['a'])):
                blur = np.load(blur_path + 'train/semiprivate/' + train_filelist[i] + '.npy')
                y_cen += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*(coordinate['a'][a-1][1]+coordinate['a'][a-1][3])*blur/2
                y_weight += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*blur/2
                x_cen += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*(coordinate['a'][a-1][2]+coordinate['a'][a-1][0])*blur/2
                x_weight += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*blur/2
            y_cen = y_cen / y_weight
            x_cen = x_cen / x_weight
            dis_cen = (y_cen-size[2])**2 + (x_cen-size[3])**2
            size_ratio = np.array([size[0]/height, size[1]/width, size[2]/height, size[3]/width, dis_cen])
            # save the head pose angle            
            np.save(position_path + 'train/semiprivate/' + train_filelist[i] + '.npy' , size_ratio)
    except ValueError:
        print('No faces detected for ' + train_filelist[i] + ". Also Dlib failed.")
        if label == 0:
            np.save(position_dataset_path + 'train/private/' + train_filelist[i] , size_ratio=np.zeros(1))
        elif label == 1:
            np.save(position_dataset_path + 'train/public/' + train_filelist[i] , size_ratio=np.zeros(1))
        else:
            np.save(position_dataset_path + 'train/semiprivate/' + train_filelist[i] , size_ratio=np.zeros(1))
        continue

00000ae5e7fcc87222f1fb6f221e7501558e5b08_0
0
00000ae5e7fcc87222f1fb6f221e7501558e5b08_1
1
00000ae5e7fcc87222f1fb6f221e7501558e5b08_2
2
00000ae5e7fcc87222f1fb6f221e7501558e5b08_3
3
00000ae5e7fcc87222f1fb6f221e7501558e5b08_4
4
00000ae5e7fcc87222f1fb6f221e7501558e5b08_5
5
00000ae5e7fcc87222f1fb6f221e7501558e5b08_6
6
00000ae5e7fcc87222f1fb6f221e7501558e5b08_7
7
00ae730b3a1b68358d4bda73978b14707cd1ef6c_0
8
00ae730b3a1b68358d4bda73978b14707cd1ef6c_1
9
00af5da9ae2bca79dd46100a485fd13bba848972_0
10
00af5da9ae2bca79dd46100a485fd13bba848972_1
11
00b1297d36c7dd6470d9439df065a10b3b241f40_0
12
00b1297d36c7dd6470d9439df065a10b3b241f40_1
13
00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b_0
14
00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b_1
15
00c3b8f6898f99bfa77bf10a1492fb959336c158_0
16
00c3b8f6898f99bfa77bf10a1492fb959336c158_1
17
00d40a80e5cbcc09883ce1ad79e79b16879786e5_0
18
00d40a80e5cbcc09883ce1ad79e79b16879786e5_1
19
00d40a80e5cbcc09883ce1ad79e79b16879786e5_2
20
00d40a80e5cbcc09883ce1ad79e79b16879786e5_3
2

29369becea4a426ddc2cb3bf56c022525e0660ef_0
178
29369becea4a426ddc2cb3bf56c022525e0660ef_1
179
2947e867a972a4b6220bde606c2f170249cf256b_0
180
2947e867a972a4b6220bde606c2f170249cf256b_1
181
2c81f92c76413555089fd066892e6f3b8d6483a6_0
182
2c81f92c76413555089fd066892e6f3b8d6483a6_1
183
30646a147f2056c06d3c2d392ec3fb345fb34769_0
184
30646a147f2056c06d3c2d392ec3fb345fb34769_1
185
313048f1954e4cef756c5e0229a422ba99c02577_0
186
313048f1954e4cef756c5e0229a422ba99c02577_1
187
313048f1954e4cef756c5e0229a422ba99c02577_2
188
313048f1954e4cef756c5e0229a422ba99c02577_3
189
315a14aa9e994c50a7de3caf28760d2067e9d7d0_0
190
315a14aa9e994c50a7de3caf28760d2067e9d7d0_1
191
315a14aa9e994c50a7de3caf28760d2067e9d7d0_2
192
315a14aa9e994c50a7de3caf28760d2067e9d7d0_3
193
337fdc05cfcd1cbac1392013c41e0897ff395a7f_0
194
337fdc05cfcd1cbac1392013c41e0897ff395a7f_1
195
337fdc05cfcd1cbac1392013c41e0897ff395a7f_2
196
337fdc05cfcd1cbac1392013c41e0897ff395a7f_3
197
36015b2612abaebd86d597b986850b54d1748e9c_0
198
36015b2612aba

7105c62c8083686f2d86b9acbf13f6e663d93b86_1
352
7105c62c8083686f2d86b9acbf13f6e663d93b86_2
353
7105c62c8083686f2d86b9acbf13f6e663d93b86_3
354
717c1ce425ddf48bb3424243243d753051c78324_0
355
717c1ce425ddf48bb3424243243d753051c78324_1
356
7238881ddb58768da739c538d87dc4ff50ab64f0_0
357
7238881ddb58768da739c538d87dc4ff50ab64f0_1
358
7238881ddb58768da739c538d87dc4ff50ab64f0_2
359
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_0
360
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_1
361
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_2
362
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_3
363
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_4
364
73542fb0b835191230401c7ea31d180ab4b28524_0
365
73542fb0b835191230401c7ea31d180ab4b28524_1
366
73542fb0b835191230401c7ea31d180ab4b28524_2
367
73542fb0b835191230401c7ea31d180ab4b28524_3
368
7509ba9c6bee9adc0addfea061b88805b3996cfe_0
369
7509ba9c6bee9adc0addfea061b88805b3996cfe_1
370
7509ba9c6bee9adc0addfea061b88805b3996cfe_2
371
7509ba9c6bee9adc0addfea061b88805b3996cfe_3
372
7509ba9c6bee9

95e23a36253798fd3e598495f7409aa0c52e6494_4
528
962699ccbba2a523b6118becfb7ed226f196c8cf_0
529
96803a7a89baf199c10c27c163fe64add772c787_0
530
96803a7a89baf199c10c27c163fe64add772c787_1
531
96803a7a89baf199c10c27c163fe64add772c787_2
532
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_0
533
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_1
534
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_2
535
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_3
536
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_4
537
9808fa72beaf6ce140e1163c9e0226633bf399ff_0
538
9808fa72beaf6ce140e1163c9e0226633bf399ff_1
539
9808fa72beaf6ce140e1163c9e0226633bf399ff_2
540
9808fa72beaf6ce140e1163c9e0226633bf399ff_3
541
9930a423044fefdb6715998e3df69ed1e00bbcda_0
542
9930a423044fefdb6715998e3df69ed1e00bbcda_1
543
9930a423044fefdb6715998e3df69ed1e00bbcda_2
544
9930a423044fefdb6715998e3df69ed1e00bbcda_3
545
9b10cffa2573ff126f6f0d1ca5494dc7031b152c_0
546
9b10cffa2573ff126f6f0d1ca5494dc7031b152c_1
547
9b10cffa2573ff126f6f0d1ca5494dc7031b152c_2
548
9b10cffa2573f

af8f5db306215ab0c19ba07625ce4da653bfe5fc_1
711
af8f5db306215ab0c19ba07625ce4da653bfe5fc_2
712
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_0
713
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_1
714
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_2
715
army_base_Google_005_0
716
army_base_Google_005_1
717
army_base_Google_007_0
718
army_base_Google_007_1
719
army_base_Google_020_0
720
army_base_Google_020_1
721
army_base_Google_021_0
722
army_base_Google_021_1
723
army_base_Google_021_2
724
army_base_Google_021_3
725
army_base_Google_028_0
726
army_base_Google_028_1
727
army_base_Google_028_2
728
army_base_Google_029_0
729
army_base_Google_032_0
730
army_base_Google_032_1
731
army_base_Google_032_2
732
army_base_Google_032_3
733
army_base_Google_045_0
734
army_base_Google_045_1
735
army_base_Google_045_2
736
army_base_Google_045_3
737
army_base_Google_045_4
738
army_base_Google_045_5
739
army_base_Google_045_6
740
army_base_Google_050_0
741
army_base_Google_050_1
742
army_base_Google_062_0
743
army_base

...

d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_1
5560
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_10
5561
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_11
5562
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_2
5563
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_3
5564
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_4
5565
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_5
5566
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_6
5567
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_7
5568
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_8
5569
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_9
5570
d97fe45ec65ce7d8959982019885ada3110f4545_0
5571
d97fe45ec65ce7d8959982019885ada3110f4545_1
5572
d97fe45ec65ce7d8959982019885ada3110f4545_10
5573
d97fe45ec65ce7d8959982019885ada3110f4545_2
5574
d97fe45ec65ce7d8959982019885ada3110f4545_3
5575
d97fe45ec65ce7d8959982019885ada3110f4545_4
5576
d97fe45ec65ce7d8959982019885ada3110f4545_5
5577
d97fe45ec65ce7d8959982019885ada3110f4545_6
5578
d97fe45ec65ce7d8959982019885ada3110f4545_7
5579
d97fe45ec65ce7d8959982019885ada3110f4

frame_5yjsc955gxc7qisp_1
5779
frame_6gmtv0nc9klfzvn8_0
5780
frame_6gmtv0nc9klfzvn8_1
5781
frame_6gmtv0nc9klfzvn8_10
5782
frame_6gmtv0nc9klfzvn8_11
5783
frame_6gmtv0nc9klfzvn8_12
5784
frame_6gmtv0nc9klfzvn8_2
5785
frame_6gmtv0nc9klfzvn8_3
5786
frame_6gmtv0nc9klfzvn8_4
5787
frame_6gmtv0nc9klfzvn8_5
5788
frame_6gmtv0nc9klfzvn8_6
5789
frame_6gmtv0nc9klfzvn8_7
5790
frame_6gmtv0nc9klfzvn8_8
5791
frame_6gmtv0nc9klfzvn8_9
5792
frame_7scy8m3m8q3qpnnv_0
5793
frame_7scy8m3m8q3qpnnv_1
5794
frame_7scy8m3m8q3qpnnv_2
5795
frame_7scy8m3m8q3qpnnv_3
5796
frame_7scy8m3m8q3qpnnv_4
5797
frame_7scy8m3m8q3qpnnv_5
5798
frame_7wyexeko49kea3y4_0
5799
frame_7wyexeko49kea3y4_1
5800
frame_7wyexeko49kea3y4_2
5801
frame_8o4lhcl4zw216bis_0
5802
frame_8o4lhcl4zw216bis_1
5803
frame_8o4lhcl4zw216bis_2
5804
frame_8o4lhcl4zw216bis_3
5805
frame_8o4lhcl4zw216bis_4
5806
frame_8o4lhcl4zw216bis_5
5807
frame_8o4lhcl4zw216bis_6
5808
frame_92xu023f264wt62w_0
5809
frame_92xu023f264wt62w_1
5810
frame_banhg5nf6zmt6xwb_0
5811
frame_b

international-students-in-class-studying-in-ireland-or-the-uk_5
6018
international-students-in-class-studying-in-ireland-or-the-uk_6
6019
international-students-in-class-studying-in-ireland-or-the-uk_7
6020
istockphoto-473222205-640x640_0
6021
istockphoto-473222205-640x640_1
6022
istockphoto-473222205-640x640_2
6023
istockphoto-473222205-640x640_3
6024
istockphoto-473222205-640x640_4
6025
laboratory_Google_008_0
6026
laboratory_Google_008_1
6027
laboratory_Google_008_2
6028
laboratory_Google_009_0
6029
laboratory_Google_009_1
6030
laboratory_Google_033_0
6031
laboratory_Google_041_0
6032
laboratory_Google_041_1
6033
laboratory_Google_041_2
6034
laboratory_Google_048_0
6035
laboratory_Google_051_0
6036
laboratory_Google_051_1
6037
laboratory_Google_054_0
6038
laboratory_Google_085_0
6039
laboratory_Google_085_1
6040
laboratory_Google_116_0
6041
laboratory_Google_116_1
6042
laboratory_Google_182_0
6043
laboratory_Google_182_1
6044
laboratory_Google_182_2
6045
laboratory_Google_205_0
6046

lecture_room_Google_432_4
6286
lecture_room_Google_437_0
6287
lecture_room_Google_445_0
6288
lecture_room_Google_459_0
6289
lecture_room_Google_459_1
6290
lecture_room_Google_473_0
6291
lecture_room_Google_473_1
6292
lecture_room_Google_473_2
6293
lecture_room_Google_473_3
6294
lecture_room_Google_473_4
6295
lecture_room_Google_473_5
6296
lecture_room_Google_473_6
6297
lecture_room_Google_473_7
6298
lecture_room_Google_493_0
6299
lecture_room_Google_493_1
6300
lecture_room_Google_493_2
6301
lecture_room_Google_493_3
6302
lecture_room_Google_517_0
6303
lecture_room_Google_517_1
6304
lecture_room_Google_517_2
6305
lecture_room_Google_517_3
6306
lecture_room_Google_540_0
6307
lecture_room_Google_540_1
6308
lecture_room_Google_540_2
6309
lecture_room_Google_540_3
6310
lecture_room_Google_540_4
6311
lecture_room_Google_549_0
6312
lecture_room_Google_549_1
6313
lecture_room_Google_549_2
6314
lecture_room_Google_562_0
6315
lecture_room_Google_562_1
6316
lecture_room_Google_562_2
6317
lecture_

6524
sun_aopdnpmxpchoblrv_2
6525
sun_aopdnpmxpchoblrv_3
6526
sun_aopdnpmxpchoblrv_4
6527
sun_aopdnpmxpchoblrv_5
6528
sun_aopdnpmxpchoblrv_6
6529
sun_aopdnpmxpchoblrv_7
6530
sun_aopdnpmxpchoblrv_8
6531
sun_aopdnpmxpchoblrv_9
6532
sun_aryxslvwejoemepu_0
6533
sun_aryxslvwejoemepu_1
6534
sun_aslguortidcunuxu_0
6535
sun_aslguortidcunuxu_1
6536
sun_aslguortidcunuxu_2
6537
sun_aslguortidcunuxu_3
6538
sun_atfeohhwoluvrdyp_0
6539
sun_atfeohhwoluvrdyp_1
6540
sun_atfeohhwoluvrdyp_2
6541
sun_atfeohhwoluvrdyp_3
6542
sun_autlunbwggaskvdb_0
6543
sun_autlunbwggaskvdb_1
6544
sun_autlunbwggaskvdb_2
6545
sun_autlunbwggaskvdb_3
6546
sun_autlunbwggaskvdb_4
6547
sun_avjotqyblnswxoio_0
6548
sun_avjotqyblnswxoio_1
6549
sun_avjotqyblnswxoio_2
6550
sun_awmodvmqonpeudqi_0
6551
sun_awmodvmqonpeudqi_1
6552
sun_awmodvmqonpeudqi_2
6553
sun_awmodvmqonpeudqi_3
6554
sun_baaiyyqorxbkepxr_0
6555
sun_baaiyyqorxbkepxr_1
6556
sun_baaiyyqorxbkepxr_2
6557
sun_baaiyyqorxbkepxr_3
6558
sun_baaiyyqorxbkepxr_4
6559
sun_baaiyyqorxb

In [27]:
for i in range(len(validation_dataset)):
    image, label = validation_dataset[i]
    print(val_filelist[i])
    print(i)
    y_cen = 0
    y_weight = 0
    x_cen = 0
    x_weight = 0
    try:
        if label == 0:            
            if os.path.isfile(position_path + 'val/private/' + val_filelist[i] + '.npy'):
                print(val_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'val/private/' + val_filelist[i] + '.jpg'
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/val/private/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            size = np.array([bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0],(bounding_boxes[1]+bounding_boxes[3])/2, (bounding_boxes[0]+bounding_boxes[2])/2])#(up, left, bottom, right)
            image = cv2.imread('./Dataset/images/val/private/' + image_name + '.jpg')
            height, width = image.shape[:2]
            for a in range(len(coordinate['a'])):
                blur = np.load(blur_path + 'val/private/' + val_filelist[i] + '.npy')
                y_cen += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*(coordinate['a'][a-1][1]+coordinate['a'][a-1][3])*blur/2
                y_weight += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*blur/2
                x_cen += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*(coordinate['a'][a-1][2]+coordinate['a'][a-1][0])*blur/2
                x_weight += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*blur/2
            y_cen = y_cen / y_weight
            x_cen = x_cen / x_weight
            dis_cen = (y_cen-size[2])**2 + (x_cen-size[3])**2
            size_ratio = np.array([size[0]/height, size[1]/width, size[2]/height, size[3]/width, dis_cen])
            # save the head pose angle            
            np.save(position_path + 'val/private/' + val_filelist[i] + '.npy' , size_ratio)

        elif label == 1:
            if os.path.isfile(position_path + 'val/public/' + val_filelist[i] + '.npy'):
                print(val_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'val/public/' + val_filelist[i] + '.jpg'
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/val/public/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            size = np.array([bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0],(bounding_boxes[1]+bounding_boxes[3])/2, (bounding_boxes[0]+bounding_boxes[2])/2])#(up, left, bottom, right)
            image = cv2.imread('./Dataset/images/val/public/' + image_name + '.jpg')
            height, width = image.shape[:2]
            for a in range(len(coordinate['a'])):
                blur = np.load(blur_path + 'val/public/' + val_filelist[i] + '.npy')
                y_cen += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*(coordinate['a'][a-1][1]+coordinate['a'][a-1][3])*blur/2
                y_weight += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*blur/2
                x_cen += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*(coordinate['a'][a-1][2]+coordinate['a'][a-1][0])*blur/2
                x_weight += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*blur/2
            y_cen = y_cen / y_weight
            x_cen = x_cen / x_weight
            dis_cen = (y_cen-size[2])**2 + (x_cen-size[3])**2
            size_ratio = np.array([size[0]/height, size[1]/width, size[2]/height, size[3]/width, dis_cen])
            # save the head pose angle            
            np.save(position_path + 'val/public/' + val_filelist[i] + '.npy' , size_ratio)
        else:
            if os.path.isfile(position_path + 'val/semiprivate/' + val_filelist[i] + '.npy'):
                print(val_filelist[i] + ' Already present')
                continue
            face_file = dataset_path + 'val/semiprivate/' + val_filelist[i] + '.jpg'
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            coordinate = np.load('./Dataset/FaceCoordinates/val/semiprivate/' + image_name + '.npz')
            bounding_boxes = coordinate['a'][face_num]
            size = np.array([bounding_boxes[3]-bounding_boxes[1],bounding_boxes[2]-bounding_boxes[0],(bounding_boxes[1]+bounding_boxes[3])/2, (bounding_boxes[0]+bounding_boxes[2])/2])#(up, left, bottom, right)
            image = cv2.imread('./Dataset/images/val/semiprivate/' + image_name + '.jpg')
            height, width = image.shape[:2]
            for a in range(len(coordinate['a'])):
                blur = np.load(blur_path + 'val/semiprivate/' + val_filelist[i] + '.npy')
                y_cen += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*(coordinate['a'][a-1][1]+coordinate['a'][a-1][3])*blur/2
                y_weight += (coordinate['a'][a-1][3]-coordinate['a'][a-1][1])*blur/2
                x_cen += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*(coordinate['a'][a-1][2]+coordinate['a'][a-1][0])*blur/2
                x_weight += (coordinate['a'][a-1][2]-coordinate['a'][a-1][0])*blur/2
            y_cen = y_cen / y_weight
            x_cen = x_cen / x_weight
            dis_cen = (y_cen-size[2])**2 + (x_cen-size[3])**2
            size_ratio = np.array([size[0]/height, size[1]/width, size[2]/height, size[3]/width, dis_cen])
            # save the head pose angle            
            np.save(position_path + 'val/semiprivate/' + val_filelist[i] + '.npy' , size_ratio)
    except ValueError:
        print('No faces detected for ' + val_filelist[i] + ". Also Dlib failed.")
        if label == 0:
            np.save(position_dataset_path + 'val/private/' + val_filelist[i] , size_ratio=np.zeros(1))
        elif label == 1:
            np.save(position_dataset_path + 'val/public/' + val_filelist[i] , size_ratio=np.zeros(1))
        else:
            np.save(position_dataset_path + 'val/semiprivate/' + val_filelist[i] , size_ratio=np.zeros(1))
        continue

04085231fc015457761e587ee53e4d1be6eb3666_0
0
04085231fc015457761e587ee53e4d1be6eb3666_0 Already present
04085231fc015457761e587ee53e4d1be6eb3666_1
1
04085231fc015457761e587ee53e4d1be6eb3666_1 Already present
04085231fc015457761e587ee53e4d1be6eb3666_2
2
04085231fc015457761e587ee53e4d1be6eb3666_2 Already present
04085231fc015457761e587ee53e4d1be6eb3666_3
3
04085231fc015457761e587ee53e4d1be6eb3666_3 Already present
11708847396_9383898125_b_0
4
11708847396_9383898125_b_0 Already present
11708847396_9383898125_b_1
5
11708847396_9383898125_b_1 Already present
12772838113_10d6e384c0_b_0
6
12772838113_10d6e384c0_b_0 Already present
12772838113_10d6e384c0_b_1
7
12772838113_10d6e384c0_b_1 Already present
12772838113_10d6e384c0_b_2
8
12772838113_10d6e384c0_b_2 Already present
12772897004_3573abfe77_b_0
9
12772897004_3573abfe77_b_0 Already present
12772897004_3573abfe77_b_1
10
12772897004_3573abfe77_b_1 Already present
12772897004_3573abfe77_b_2
11
12772897004_3573abfe77_b_2 Already present
127728

395645143_92392a8b70_144_12943170@N00_1 Already present
395645143_92392a8b70_144_12943170@N00_2
96
395645143_92392a8b70_144_12943170@N00_2 Already present
395645143_92392a8b70_144_12943170@N00_3
97
395645143_92392a8b70_144_12943170@N00_3 Already present
395645143_92392a8b70_144_12943170@N00_4
98
395645143_92392a8b70_144_12943170@N00_4 Already present
395645143_92392a8b70_144_12943170@N00_5
99
395645143_92392a8b70_144_12943170@N00_5 Already present
42b0bfe84497a07a50ca13fb95f2ee1339c99b52_0
100
42b0bfe84497a07a50ca13fb95f2ee1339c99b52_0 Already present
42b0bfe84497a07a50ca13fb95f2ee1339c99b52_1
101
42b0bfe84497a07a50ca13fb95f2ee1339c99b52_1 Already present
4635c0fd6368e4d764bc8cca49f1733094275fab_0
102
4635c0fd6368e4d764bc8cca49f1733094275fab_0 Already present
4635c0fd6368e4d764bc8cca49f1733094275fab_1
103
4635c0fd6368e4d764bc8cca49f1733094275fab_1 Already present
4635c0fd6368e4d764bc8cca49f1733094275fab_2
104
4635c0fd6368e4d764bc8cca49f1733094275fab_2 Already present
52040edf602bf_0
10

188
a38cdb897ef104383634b53c279849019bc217a9_3 Already present
army_base_Google_003_0
189
army_base_Google_003_0 Already present
army_base_Google_003_1
190
army_base_Google_003_1 Already present
army_base_Google_003_10
191
army_base_Google_003_10 Already present
army_base_Google_003_11
192
army_base_Google_003_11 Already present
army_base_Google_003_12
193
army_base_Google_003_12 Already present
army_base_Google_003_13
194
army_base_Google_003_13 Already present
army_base_Google_003_14
195
army_base_Google_003_14 Already present
army_base_Google_003_15
196
army_base_Google_003_15 Already present
army_base_Google_003_16
197
army_base_Google_003_16 Already present
army_base_Google_003_17
198
army_base_Google_003_17 Already present
army_base_Google_003_18
199
army_base_Google_003_18 Already present
army_base_Google_003_19
200
army_base_Google_003_19 Already present
army_base_Google_003_2
201
army_base_Google_003_2 Already present
army_base_Google_003_20
202
army_base_Google_003_20 Already

fd73023c95f107f4e453138996b168be13de6849_1 Already present
feab80012375d75c813d0095c41d8d633a1e1c5b_0
282
feab80012375d75c813d0095c41d8d633a1e1c5b_0 Already present
feab80012375d75c813d0095c41d8d633a1e1c5b_1
283
feab80012375d75c813d0095c41d8d633a1e1c5b_1 Already present
feab80012375d75c813d0095c41d8d633a1e1c5b_2
284
feab80012375d75c813d0095c41d8d633a1e1c5b_2 Already present
feab80012375d75c813d0095c41d8d633a1e1c5b_3
285
feab80012375d75c813d0095c41d8d633a1e1c5b_3 Already present
frame_1egig72uz4nhprvo_0
286
frame_1egig72uz4nhprvo_0 Already present
frame_1egig72uz4nhprvo_1
287
frame_1egig72uz4nhprvo_1 Already present
frame_1egig72uz4nhprvo_10
288
frame_1egig72uz4nhprvo_10 Already present
frame_1egig72uz4nhprvo_11
289
frame_1egig72uz4nhprvo_11 Already present
frame_1egig72uz4nhprvo_12
290
frame_1egig72uz4nhprvo_12 Already present
frame_1egig72uz4nhprvo_13
291
frame_1egig72uz4nhprvo_13 Already present
frame_1egig72uz4nhprvo_14
292
frame_1egig72uz4nhprvo_14 Already present
frame_1egig72uz4n

33474103-high-school-students-working-on-campus-with-teacher_4
375
33474103-high-school-students-working-on-campus-with-teacher_4 Already present
33474103-high-school-students-working-on-campus-with-teacher_5
376
33474103-high-school-students-working-on-campus-with-teacher_5 Already present
33474103-high-school-students-working-on-campus-with-teacher_6
377
33474103-high-school-students-working-on-campus-with-teacher_6 Already present
33474103-high-school-students-working-on-campus-with-teacher_7
378
33474103-high-school-students-working-on-campus-with-teacher_7 Already present
33474103-high-school-students-working-on-campus-with-teacher_8
379
33474103-high-school-students-working-on-campus-with-teacher_8 Already present
33474103-high-school-students-working-on-campus-with-teacher_9
380
33474103-high-school-students-working-on-campus-with-teacher_9 Already present
33474153-high-school-students-hanging-out-on-campus_0
381
33474153-high-school-students-hanging-out-on-campus_0 Already pres

...

519
8dbe95bce25cac4617424912284440c012f93fca_1 Already present
8dbe95bce25cac4617424912284440c012f93fca_2
520
8dbe95bce25cac4617424912284440c012f93fca_2 Already present
91d88efcc8715b039b672f769dd5f31b2ebd6d17_0
521
91d88efcc8715b039b672f769dd5f31b2ebd6d17_0 Already present
91d88efcc8715b039b672f769dd5f31b2ebd6d17_1
522
91d88efcc8715b039b672f769dd5f31b2ebd6d17_1 Already present
91d88efcc8715b039b672f769dd5f31b2ebd6d17_2
523
91d88efcc8715b039b672f769dd5f31b2ebd6d17_2 Already present
91d88efcc8715b039b672f769dd5f31b2ebd6d17_3
524
91d88efcc8715b039b672f769dd5f31b2ebd6d17_3 Already present
920x920_0
525
920x920_0 Already present
920x920_1
526
920x920_1 Already present
920x920_2
527
920x920_2 Already present
9cc41d76e44e2c9bec3bf121e8ec822efa8087a4_0
528
9cc41d76e44e2c9bec3bf121e8ec822efa8087a4_0 Already present
9cc41d76e44e2c9bec3bf121e8ec822efa8087a4_1
529
9cc41d76e44e2c9bec3bf121e8ec822efa8087a4_1 Already present
9cc41d76e44e2c9bec3bf121e8ec822efa8087a4_2
530
9cc41d76e44e2c9bec3bf121e8ec

f08130d6ff3afff07665c7c40e63c7e25d02fb75_1
607
f08130d6ff3afff07665c7c40e63c7e25d02fb75_1 Already present
f08130d6ff3afff07665c7c40e63c7e25d02fb75_2
608
f08130d6ff3afff07665c7c40e63c7e25d02fb75_2 Already present
f08130d6ff3afff07665c7c40e63c7e25d02fb75_3
609
f08130d6ff3afff07665c7c40e63c7e25d02fb75_3 Already present
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_0
610
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_0 Already present
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_1
611
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_1 Already present
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_2
612
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_2 Already present
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_3
613
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_3 Already present
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_4
614
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_4 Already present
f263409766b95af3487dfeeaf8eb8e98bd671df6_0
615
f263409766b95af3487dfeeaf8eb8e98bd671df6_0 Already present
f263409766b95af3487dfeeaf8eb8e98bd671df6_1
616

18551421-students-studying-with-laptop-computer-in-class-in-university_3
731
18551421-students-studying-with-laptop-computer-in-class-in-university_4
732
18551421-students-studying-with-laptop-computer-in-class-in-university_5
733
18551421-students-studying-with-laptop-computer-in-class-in-university_6
734
18551421-students-studying-with-laptop-computer-in-class-in-university_7
735
18551421-students-studying-with-laptop-computer-in-class-in-university_8
736
200309822-001_0
737
200309822-001_1
738
20150625_0028_0
739
20150625_0028_1
740
20150625_0028_2
741
20150625_0028_3
742
20150625_0028_4
743
20235289-group-of-high-school-students-studying-in-classroom_0
744
20235289-group-of-high-school-students-studying-in-classroom_1
745
20235289-group-of-high-school-students-studying-in-classroom_2
746
20235289-group-of-high-school-students-studying-in-classroom_3
747
20235293-smiling-senior-high-school-teacher-talking-to-student-in-classroom_0
748
20235293-smiling-senior-high-school-teacher-talk

708f50009a2971e700c6ac54e7385be8e7c705b0_1
876
708f50009a2971e700c6ac54e7385be8e7c705b0_2
877
7536b41fbeb249ddad1f91701e6c610bf9a05ca8_0
878
7536b41fbeb249ddad1f91701e6c610bf9a05ca8_1
879
7536b41fbeb249ddad1f91701e6c610bf9a05ca8_2
880
7536b41fbeb249ddad1f91701e6c610bf9a05ca8_3
881
756b95becc61c2af7afb39903e688dfffb1fe5b4_0
882
756b95becc61c2af7afb39903e688dfffb1fe5b4_1
883
756b95becc61c2af7afb39903e688dfffb1fe5b4_2
884
756b95becc61c2af7afb39903e688dfffb1fe5b4_3
885
756b95becc61c2af7afb39903e688dfffb1fe5b4_4
886
76693d316948f478cb6ef250fc214e924d4ff28f_0
887
76693d316948f478cb6ef250fc214e924d4ff28f_1
888
76693d316948f478cb6ef250fc214e924d4ff28f_2
889
76693d316948f478cb6ef250fc214e924d4ff28f_3
890
78b1ada4068d7f1ed8fc3c5e87532a0e236fc74f_0
891
78b1ada4068d7f1ed8fc3c5e87532a0e236fc74f_1
892
78b1ada4068d7f1ed8fc3c5e87532a0e236fc74f_10
893
78b1ada4068d7f1ed8fc3c5e87532a0e236fc74f_11
894
78b1ada4068d7f1ed8fc3c5e87532a0e236fc74f_12
895
78b1ada4068d7f1ed8fc3c5e87532a0e236fc74f_13
896
78b1ada40

df8592ab8ea564eb15f592ea64974414aa6a4247_1
1070
df8592ab8ea564eb15f592ea64974414aa6a4247_2
1071
df8592ab8ea564eb15f592ea64974414aa6a4247_3
1072
df8592ab8ea564eb15f592ea64974414aa6a4247_4
1073
downtown_339_0
1074
downtown_339_1
1075
downtown_339_2
1076
downtown_439_0
1077
downtown_439_1
1078
downtown_491_0
1079
downtown_714_0
1080
downtown_714_1
1081
downtown_714_2
1082
e7e6e8d420c39b7c94abfa3f44db012bc75ca953_0
1083
e7e6e8d420c39b7c94abfa3f44db012bc75ca953_1
1084
e7e6e8d420c39b7c94abfa3f44db012bc75ca953_2
1085
e7e6e8d420c39b7c94abfa3f44db012bc75ca953_3
1086
e7e6e8d420c39b7c94abfa3f44db012bc75ca953_4
1087
e81756bb421b07ac06a7404e9a8cff28942e7d5a_0
1088
e81756bb421b07ac06a7404e9a8cff28942e7d5a_1
1089
e81756bb421b07ac06a7404e9a8cff28942e7d5a_2
1090
ed07c32ef8875fe22846a1e5c04c2189dad7b6a2_0
1091
ed07c32ef8875fe22846a1e5c04c2189dad7b6a2_1
1092
ed07c32ef8875fe22846a1e5c04c2189dad7b6a2_2
1093
ed07c32ef8875fe22846a1e5c04c2189dad7b6a2_3
1094
edf2aebc42dc93ec96f788a34e41b2d2588914d0_0
1095
edf2

# Concatenate

In [19]:
for i in range(len(training_dataset)):
    image, label = training_dataset[i]
    print(train_filelist[i])
    print(i)
    if label == 0:            
        if os.path.isfile(input_path + 'train/' + train_filelist[i] + '.npy'):
            print(train_filelist[i] + ' Already present')
            continue
        #pose_path = './Dataset/inputvector/headpose/'
        bluriness = np.load(blur_path + 'train/private/' + train_filelist[i] + '.npy')
        position = np.load(position_path + 'train/private/' + train_filelist[i] + '.npy')
        inputs = np.concatenate((bluriness, position), axis=None)      
        np.save(input_path + 'train/' + train_filelist[i] + '.npy' , inputs)

    elif label == 1:
        if os.path.isfile(input_path + 'train/' + train_filelist[i] + '.npy'):
            print(train_filelist[i] + ' Already present')
            continue
        #pose_path = './Dataset/inputvector/headpose/'
        bluriness = np.load(blur_path + 'train/public/' + train_filelist[i] + '.npy')
        position = np.load(position_path + 'train/public/' + train_filelist[i] + '.npy')
        inputs = np.concatenate((bluriness, position), axis=None)       
        np.save(input_path + 'train/' + train_filelist[i] + '.npy' , inputs)
    else:
        if os.path.isfile(input_path + 'train/' + train_filelist[i] + '.npy'):
            print(train_filelist[i] + ' Already present')
            continue
        #pose_path = './Dataset/inputvector/headpose/'
        bluriness = np.load(blur_path + 'train/semiprivate/' + train_filelist[i] + '.npy')
        position = np.load(position_path + 'train/semiprivate/' + train_filelist[i] + '.npy')
        inputs = np.concatenate((bluriness, position), axis=None)       
        np.save(input_path + 'train/' + train_filelist[i] + '.npy' , inputs)

00000ae5e7fcc87222f1fb6f221e7501558e5b08_0
0
00000ae5e7fcc87222f1fb6f221e7501558e5b08_1
1
00000ae5e7fcc87222f1fb6f221e7501558e5b08_2
2
00000ae5e7fcc87222f1fb6f221e7501558e5b08_3
3
00000ae5e7fcc87222f1fb6f221e7501558e5b08_4
4
00000ae5e7fcc87222f1fb6f221e7501558e5b08_5
5
00000ae5e7fcc87222f1fb6f221e7501558e5b08_6
6
00000ae5e7fcc87222f1fb6f221e7501558e5b08_7
7
00ae730b3a1b68358d4bda73978b14707cd1ef6c_0
8
00ae730b3a1b68358d4bda73978b14707cd1ef6c_1
9
00af5da9ae2bca79dd46100a485fd13bba848972_0
10
00af5da9ae2bca79dd46100a485fd13bba848972_1
11
00b1297d36c7dd6470d9439df065a10b3b241f40_0
12
00b1297d36c7dd6470d9439df065a10b3b241f40_1
13
00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b_0
14
00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b_1
15
00c3b8f6898f99bfa77bf10a1492fb959336c158_0
16
00c3b8f6898f99bfa77bf10a1492fb959336c158_1
17
00d40a80e5cbcc09883ce1ad79e79b16879786e5_0
18
00d40a80e5cbcc09883ce1ad79e79b16879786e5_1
19
00d40a80e5cbcc09883ce1ad79e79b16879786e5_2
20
00d40a80e5cbcc09883ce1ad79e79b16879786e5_3
2

29369becea4a426ddc2cb3bf56c022525e0660ef_0
178
29369becea4a426ddc2cb3bf56c022525e0660ef_1
179
2947e867a972a4b6220bde606c2f170249cf256b_0
180
2947e867a972a4b6220bde606c2f170249cf256b_1
181
2c81f92c76413555089fd066892e6f3b8d6483a6_0
182
2c81f92c76413555089fd066892e6f3b8d6483a6_1
183
30646a147f2056c06d3c2d392ec3fb345fb34769_0
184
30646a147f2056c06d3c2d392ec3fb345fb34769_1
185
313048f1954e4cef756c5e0229a422ba99c02577_0
186
313048f1954e4cef756c5e0229a422ba99c02577_1
187
313048f1954e4cef756c5e0229a422ba99c02577_2
188
313048f1954e4cef756c5e0229a422ba99c02577_3
189
315a14aa9e994c50a7de3caf28760d2067e9d7d0_0
190
315a14aa9e994c50a7de3caf28760d2067e9d7d0_1
191
315a14aa9e994c50a7de3caf28760d2067e9d7d0_2
192
315a14aa9e994c50a7de3caf28760d2067e9d7d0_3
193
337fdc05cfcd1cbac1392013c41e0897ff395a7f_0
194
337fdc05cfcd1cbac1392013c41e0897ff395a7f_1
195
337fdc05cfcd1cbac1392013c41e0897ff395a7f_2
196
337fdc05cfcd1cbac1392013c41e0897ff395a7f_3
197
36015b2612abaebd86d597b986850b54d1748e9c_0
198
36015b2612aba

7105c62c8083686f2d86b9acbf13f6e663d93b86_1
352
7105c62c8083686f2d86b9acbf13f6e663d93b86_2
353
7105c62c8083686f2d86b9acbf13f6e663d93b86_3
354
717c1ce425ddf48bb3424243243d753051c78324_0
355
717c1ce425ddf48bb3424243243d753051c78324_1
356
7238881ddb58768da739c538d87dc4ff50ab64f0_0
357
7238881ddb58768da739c538d87dc4ff50ab64f0_1
358
7238881ddb58768da739c538d87dc4ff50ab64f0_2
359
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_0
360
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_1
361
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_2
362
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_3
363
72b8ec7c3ce38fdb69b2c7f3cb0b8baeb2d40ba1_4
364
73542fb0b835191230401c7ea31d180ab4b28524_0
365
73542fb0b835191230401c7ea31d180ab4b28524_1
366
73542fb0b835191230401c7ea31d180ab4b28524_2
367
73542fb0b835191230401c7ea31d180ab4b28524_3
368
7509ba9c6bee9adc0addfea061b88805b3996cfe_0
369
7509ba9c6bee9adc0addfea061b88805b3996cfe_1
370
7509ba9c6bee9adc0addfea061b88805b3996cfe_2
371
7509ba9c6bee9adc0addfea061b88805b3996cfe_3
372
7509ba9c6bee9

95e23a36253798fd3e598495f7409aa0c52e6494_4
528
962699ccbba2a523b6118becfb7ed226f196c8cf_0
529
96803a7a89baf199c10c27c163fe64add772c787_0
530
96803a7a89baf199c10c27c163fe64add772c787_1
531
96803a7a89baf199c10c27c163fe64add772c787_2
532
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_0
533
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_1
534
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_2
535
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_3
536
97f5bbc5d6d44ca67715f2880d466b2c0737cd4b_4
537
9808fa72beaf6ce140e1163c9e0226633bf399ff_0
538
9808fa72beaf6ce140e1163c9e0226633bf399ff_1
539
9808fa72beaf6ce140e1163c9e0226633bf399ff_2
540
9808fa72beaf6ce140e1163c9e0226633bf399ff_3
541
9930a423044fefdb6715998e3df69ed1e00bbcda_0
542
9930a423044fefdb6715998e3df69ed1e00bbcda_1
543
9930a423044fefdb6715998e3df69ed1e00bbcda_2
544
9930a423044fefdb6715998e3df69ed1e00bbcda_3
545
9b10cffa2573ff126f6f0d1ca5494dc7031b152c_0
546
9b10cffa2573ff126f6f0d1ca5494dc7031b152c_1
547
9b10cffa2573ff126f6f0d1ca5494dc7031b152c_2
548
9b10cffa2573f

af8f5db306215ab0c19ba07625ce4da653bfe5fc_1
711
af8f5db306215ab0c19ba07625ce4da653bfe5fc_2
712
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_0
713
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_1
714
afeabf3aef32d85b2b0f0610a9abb146210dbb6e_2
715
army_base_Google_005_0
716
army_base_Google_005_1
717
army_base_Google_007_0
718
army_base_Google_007_1
719
army_base_Google_020_0
720
army_base_Google_020_1
721
army_base_Google_021_0
722
army_base_Google_021_1
723
army_base_Google_021_2
724
army_base_Google_021_3
725
army_base_Google_028_0
726
army_base_Google_028_1
727
army_base_Google_028_2
728
army_base_Google_029_0
729
army_base_Google_032_0
730
army_base_Google_032_1
731
army_base_Google_032_2
732
army_base_Google_032_3
733
army_base_Google_045_0
734
army_base_Google_045_1
735
army_base_Google_045_2
736
army_base_Google_045_3
737
army_base_Google_045_4
738
army_base_Google_045_5
739
army_base_Google_045_6
740
army_base_Google_050_0
741
army_base_Google_050_1
742
army_base_Google_062_0
743
army_base

...

d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_1
5560
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_10
5561
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_11
5562
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_2
5563
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_3
5564
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_4
5565
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_5
5566
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_6
5567
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_7
5568
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_8
5569
d8c1bbb00a42e98e70eb108ed263e16cc9acb2e4_9
5570
d97fe45ec65ce7d8959982019885ada3110f4545_0
5571
d97fe45ec65ce7d8959982019885ada3110f4545_1
5572
d97fe45ec65ce7d8959982019885ada3110f4545_10
5573
d97fe45ec65ce7d8959982019885ada3110f4545_2
5574
d97fe45ec65ce7d8959982019885ada3110f4545_3
5575
d97fe45ec65ce7d8959982019885ada3110f4545_4
5576
d97fe45ec65ce7d8959982019885ada3110f4545_5
5577
d97fe45ec65ce7d8959982019885ada3110f4545_6
5578
d97fe45ec65ce7d8959982019885ada3110f4545_7
5579
d97fe45ec65ce7d8959982019885ada3110f4

frame_5yjsc955gxc7qisp_1
5779
frame_6gmtv0nc9klfzvn8_0
5780
frame_6gmtv0nc9klfzvn8_1
5781
frame_6gmtv0nc9klfzvn8_10
5782
frame_6gmtv0nc9klfzvn8_11
5783
frame_6gmtv0nc9klfzvn8_12
5784
frame_6gmtv0nc9klfzvn8_2
5785
frame_6gmtv0nc9klfzvn8_3
5786
frame_6gmtv0nc9klfzvn8_4
5787
frame_6gmtv0nc9klfzvn8_5
5788
frame_6gmtv0nc9klfzvn8_6
5789
frame_6gmtv0nc9klfzvn8_7
5790
frame_6gmtv0nc9klfzvn8_8
5791
frame_6gmtv0nc9klfzvn8_9
5792
frame_7scy8m3m8q3qpnnv_0
5793
frame_7scy8m3m8q3qpnnv_1
5794
frame_7scy8m3m8q3qpnnv_2
5795
frame_7scy8m3m8q3qpnnv_3
5796
frame_7scy8m3m8q3qpnnv_4
5797
frame_7scy8m3m8q3qpnnv_5
5798
frame_7wyexeko49kea3y4_0
5799
frame_7wyexeko49kea3y4_1
5800
frame_7wyexeko49kea3y4_2
5801
frame_8o4lhcl4zw216bis_0
5802
frame_8o4lhcl4zw216bis_1
5803
frame_8o4lhcl4zw216bis_2
5804
frame_8o4lhcl4zw216bis_3
5805
frame_8o4lhcl4zw216bis_4
5806
frame_8o4lhcl4zw216bis_5
5807
frame_8o4lhcl4zw216bis_6
5808
frame_92xu023f264wt62w_0
5809
frame_92xu023f264wt62w_1
5810
frame_banhg5nf6zmt6xwb_0
5811
frame_b

international-students-in-class-studying-in-ireland-or-the-uk_5
6018
international-students-in-class-studying-in-ireland-or-the-uk_6
6019
international-students-in-class-studying-in-ireland-or-the-uk_7
6020
istockphoto-473222205-640x640_0
6021
istockphoto-473222205-640x640_1
6022
istockphoto-473222205-640x640_2
6023
istockphoto-473222205-640x640_3
6024
istockphoto-473222205-640x640_4
6025
laboratory_Google_008_0
6026
laboratory_Google_008_1
6027
laboratory_Google_008_2
6028
laboratory_Google_009_0
6029
laboratory_Google_009_1
6030
laboratory_Google_033_0
6031
laboratory_Google_041_0
6032
laboratory_Google_041_1
6033
laboratory_Google_041_2
6034
laboratory_Google_048_0
6035
laboratory_Google_051_0
6036
laboratory_Google_051_1
6037
laboratory_Google_054_0
6038
laboratory_Google_085_0
6039
laboratory_Google_085_1
6040
laboratory_Google_116_0
6041
laboratory_Google_116_1
6042
laboratory_Google_182_0
6043
laboratory_Google_182_1
6044
laboratory_Google_182_2
6045
laboratory_Google_205_0
6046

lecture_room_Google_432_4
6286
lecture_room_Google_437_0
6287
lecture_room_Google_445_0
6288
lecture_room_Google_459_0
6289
lecture_room_Google_459_1
6290
lecture_room_Google_473_0
6291
lecture_room_Google_473_1
6292
lecture_room_Google_473_2
6293
lecture_room_Google_473_3
6294
lecture_room_Google_473_4
6295
lecture_room_Google_473_5
6296
lecture_room_Google_473_6
6297
lecture_room_Google_473_7
6298
lecture_room_Google_493_0
6299
lecture_room_Google_493_1
6300
lecture_room_Google_493_2
6301
lecture_room_Google_493_3
6302
lecture_room_Google_517_0
6303
lecture_room_Google_517_1
6304
lecture_room_Google_517_2
6305
lecture_room_Google_517_3
6306
lecture_room_Google_540_0
6307
lecture_room_Google_540_1
6308
lecture_room_Google_540_2
6309
lecture_room_Google_540_3
6310
lecture_room_Google_540_4
6311
lecture_room_Google_549_0
6312
lecture_room_Google_549_1
6313
lecture_room_Google_549_2
6314
lecture_room_Google_562_0
6315
lecture_room_Google_562_1
6316
lecture_room_Google_562_2
6317
lecture_

sun_aopdnpmxpchoblrv_11
6518
sun_aopdnpmxpchoblrv_12
6519
sun_aopdnpmxpchoblrv_13
6520
sun_aopdnpmxpchoblrv_14
6521
sun_aopdnpmxpchoblrv_15
6522
sun_aopdnpmxpchoblrv_16
6523
sun_aopdnpmxpchoblrv_17
6524
sun_aopdnpmxpchoblrv_2
6525
sun_aopdnpmxpchoblrv_3
6526
sun_aopdnpmxpchoblrv_4
6527
sun_aopdnpmxpchoblrv_5
6528
sun_aopdnpmxpchoblrv_6
6529
sun_aopdnpmxpchoblrv_7
6530
sun_aopdnpmxpchoblrv_8
6531
sun_aopdnpmxpchoblrv_9
6532
sun_aryxslvwejoemepu_0
6533
sun_aryxslvwejoemepu_1
6534
sun_aslguortidcunuxu_0
6535
sun_aslguortidcunuxu_1
6536
sun_aslguortidcunuxu_2
6537
sun_aslguortidcunuxu_3
6538
sun_atfeohhwoluvrdyp_0
6539
sun_atfeohhwoluvrdyp_1
6540
sun_atfeohhwoluvrdyp_2
6541
sun_atfeohhwoluvrdyp_3
6542
sun_autlunbwggaskvdb_0
6543
sun_autlunbwggaskvdb_1
6544
sun_autlunbwggaskvdb_2
6545
sun_autlunbwggaskvdb_3
6546
sun_autlunbwggaskvdb_4
6547
sun_avjotqyblnswxoio_0
6548
sun_avjotqyblnswxoio_1
6549
sun_avjotqyblnswxoio_2
6550
sun_awmodvmqonpeudqi_0
6551
sun_awmodvmqonpeudqi_1
6552
sun_awmodvmqo

In [87]:
for i in range(len(validation_dataset)):
    image, label = validation_dataset[i]
    print(val_filelist[i])
    print(i)
    if label == 0:            
        if os.path.isfile(input_path + '/val/' + val_filelist[i] + '.npy'):
            print(val_filelist[i] + ' Already present')
            continue
        #pose_path = './Dataset/inputvector/headpose/'
        bluriness = np.load(blur_path + 'val/private/' + val_filelist[i] + '.npy')
        position = np.load(position_path + 'val/private/' + val_filelist[i] + '.npy')
        inputs = np.concatenate((bluriness, position), axis=None)      
        np.save(input_path + '/val/' + val_filelist[i] + '.npy' , inputs)

    elif label == 1:
        if os.path.isfile(input_path + 'val/' + val_filelist[i] + '.npy'):
            print(val_filelist[i] + ' Already present')
            continue
        #pose_path = './Dataset/inputvector/headpose/'
        bluriness = np.load(blur_path + 'val/public/' + val_filelist[i] + '.npy')
        position = np.load(position_path + 'val/public/' + val_filelist[i] + '.npy')
        inputs = np.concatenate((bluriness, position), axis=None)       
        np.save(input_path + '/val/' + val_filelist[i] + '.npy' , inputs)
    else:
        if os.path.isfile(input_path + '/val/' + val_filelist[i] + '.npy'):
            print(val_filelist[i] + ' Already present')
            continue
        #pose_path = './Dataset/inputvector/headpose/'
        bluriness = np.load(blur_path + 'val/semiprivate/' + val_filelist[i] + '.npy')
        position = np.load(position_path + 'val/semiprivate/' + val_filelist[i] + '.npy')
        inputs = np.concatenate((bluriness, position), axis=None)       
        np.save(input_path + '/val/' + val_filelist[i] + '.npy' , inputs)

04085231fc015457761e587ee53e4d1be6eb3666_0
0
04085231fc015457761e587ee53e4d1be6eb3666_1
1
04085231fc015457761e587ee53e4d1be6eb3666_2
2
04085231fc015457761e587ee53e4d1be6eb3666_3
3
11708847396_9383898125_b_0
4
11708847396_9383898125_b_1
5
12772838113_10d6e384c0_b_0
6
12772838113_10d6e384c0_b_1
7
12772838113_10d6e384c0_b_2
8
12772897004_3573abfe77_b_0
9
12772897004_3573abfe77_b_1
10
12772897004_3573abfe77_b_2
11
12772897004_3573abfe77_b_3
12
12772897004_3573abfe77_b_4
13
137300_0
14
137300_1
15
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_0
16
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_1
17
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_2
18
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_3
19
13cfa59c9fbc82b5cc6866c4d525f35bd1003777_4
20
143eb889d808e38195c2ac1aa20139e285905aa0_0
21
143eb889d808e38195c2ac1aa20139e285905aa0_1
22
143eb889d808e38195c2ac1aa20139e285905aa0_2
23
143eb889d808e38195c2ac1aa20139e285905aa0_3
24
14938114374_89dd0c0566_b_0
25
14938114374_89dd0c0566_b_1
26
14938114374_89dd0c0566_b_2
2

c5438f7d3274bcd940cabbdeb347f6989daa7fa7_2
229
c5438f7d3274bcd940cabbdeb347f6989daa7fa7_3
230
c5438f7d3274bcd940cabbdeb347f6989daa7fa7_4
231
c6be8124ca17d65a3918c792067ef25405014c53_0
232
c6be8124ca17d65a3918c792067ef25405014c53_1
233
c6be8124ca17d65a3918c792067ef25405014c53_2
234
c6be8124ca17d65a3918c792067ef25405014c53_3
235
coffee_shop_272_0
236
coffee_shop_272_1
237
d551eea664a51a9da59bbe388629e81e53748277_0
238
d551eea664a51a9da59bbe388629e81e53748277_1
239
d551eea664a51a9da59bbe388629e81e53748277_2
240
dc1acce8b9c1cc2b74c962b082b9f1956951e83f_0
241
dc1acce8b9c1cc2b74c962b082b9f1956951e83f_1
242
dc1acce8b9c1cc2b74c962b082b9f1956951e83f_2
243
dc1acce8b9c1cc2b74c962b082b9f1956951e83f_3
244
de87f817530cdadf7ce41acfdce40e02b7933d2f_0
245
de87f817530cdadf7ce41acfdce40e02b7933d2f_1
246
de87f817530cdadf7ce41acfdce40e02b7933d2f_2
247
de87f817530cdadf7ce41acfdce40e02b7933d2f_3
248
de87f817530cdadf7ce41acfdce40e02b7933d2f_4
249
downtown_136_0
250
downtown_136_1
251
drug_store_Google_285_0
2

33d21955d18d74715d844b9bae2ce0d0bf5dcfc2_2
411
366577_orig_0
412
366577_orig_1
413
366577_orig_2
414
366577_orig_3
415
366577_orig_4
416
366577_orig_5
417
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_0
418
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_1
419
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_2
420
366C209A00000578-3698055-Lady_Marina_and_sister_Lady_Amelia_Windsor_attended_the_Queen_s_-a-2_1468998228913_3
421
49dae06853e5650bf3862ffcc5ca12c28f30bbc2_0
422
49dae06853e5650bf3862ffcc5ca12c28f30bbc2_1
423
50a4cddc5bfa2709f962f8eb13459739adc6e2b6_0
424
50a4cddc5bfa2709f962f8eb13459739adc6e2b6_1
425
50e5269163e82037e475842fdb28113894ce2c80_0
426
50e5269163e82037e475842fdb28113894ce2c80_1
427
50e5269163e82037e475842fdb28113894ce2c80_2
428
53a40ea9e0fb33b56ff4088c1a60b03dc4c14cd7_0
429
53a40ea9e0fb3

e29822cab99fb862a58ddf5802ff9e7019d04ffb_0
598
e29822cab99fb862a58ddf5802ff9e7019d04ffb_1
599
e29822cab99fb862a58ddf5802ff9e7019d04ffb_2
600
e362ea15b3b68406a4644efcc60fbe802af66014_0
601
e362ea15b3b68406a4644efcc60fbe802af66014_1
602
ecaaa1115d45f1c5d45cfbcf185f98b0ba19c07a_0
603
ecaaa1115d45f1c5d45cfbcf185f98b0ba19c07a_1
604
ecaaa1115d45f1c5d45cfbcf185f98b0ba19c07a_2
605
f08130d6ff3afff07665c7c40e63c7e25d02fb75_0
606
f08130d6ff3afff07665c7c40e63c7e25d02fb75_1
607
f08130d6ff3afff07665c7c40e63c7e25d02fb75_2
608
f08130d6ff3afff07665c7c40e63c7e25d02fb75_3
609
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_0
610
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_1
611
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_2
612
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_3
613
f1f0e255cb14e6c0233951e220377f0fae9ec7b3_4
614
f263409766b95af3487dfeeaf8eb8e98bd671df6_0
615
f263409766b95af3487dfeeaf8eb8e98bd671df6_1
616
f263409766b95af3487dfeeaf8eb8e98bd671df6_2
617
f263409766b95af3487dfeeaf8eb8e98bd671df6_3
618
f32f39a28a3cb

25216282-asian-student-raising-her-hand-in-a-university-classroom_6
777
25216282-asian-student-raising-her-hand-in-a-university-classroom_7
778
25216282-asian-student-raising-her-hand-in-a-university-classroom_8
779
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_0
780
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_1
781
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_2
782
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_3
783
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_4
784
26_02_2016_18_10_11_dq2d8jt2aa7fratlorikkhtdh5_wdmscly8ig_5
785
290e64555c20ba308adb8f7a7867f6b15be31475_0
786
290e64555c20ba308adb8f7a7867f6b15be31475_1
787
29af9a009bfd6501f625ea926bd630bc2047b572_0
788
29af9a009bfd6501f625ea926bd630bc2047b572_1
789
29af9a009bfd6501f625ea926bd630bc2047b572_2
790
29af9a009bfd6501f625ea926bd630bc2047b572_3
791
30505970f0e1b25ab0a3a1459e97505b09f60dd7_0
792
30505970f0e1b25ab0a3a1459e97505b09f60dd7_1
793
30505970f0e1b25ab0a3a145

91e595030162b17f0053281f622c4b05f689a693_3
933
922aabfe985ebd5a167421e79e059846b9c34147_0
934
922aabfe985ebd5a167421e79e059846b9c34147_1
935
96bb663e0e0c4e10276b358d39d62e35f57a1d1e_0
936
96bb663e0e0c4e10276b358d39d62e35f57a1d1e_1
937
96bb663e0e0c4e10276b358d39d62e35f57a1d1e_2
938
96bb663e0e0c4e10276b358d39d62e35f57a1d1e_3
939
99b417f405d60fbe05f175b6543fa3a5196353ef_0
940
99b417f405d60fbe05f175b6543fa3a5196353ef_1
941
9f7dbf0eb42fd35ad17c8b5ee6c4faf5f358f1fe_0
942
9f7dbf0eb42fd35ad17c8b5ee6c4faf5f358f1fe_1
943
9f7dbf0eb42fd35ad17c8b5ee6c4faf5f358f1fe_2
944
b0f749141413253025bd25a56695a2f7c53c1e2e_0
945
b0f749141413253025bd25a56695a2f7c53c1e2e_1
946
b0f749141413253025bd25a56695a2f7c53c1e2e_2
947
b0f749141413253025bd25a56695a2f7c53c1e2e_3
948
b0f749141413253025bd25a56695a2f7c53c1e2e_4
949
b1e0a816bb41f3c3589686b12d17702dbc577a40_0
950
b1e0a816bb41f3c3589686b12d17702dbc577a40_1
951
b1e0a816bb41f3c3589686b12d17702dbc577a40_2
952
b1e0a816bb41f3c3589686b12d17702dbc577a40_3
953
bd1466e930fcf

public_transportation_028_4
1163
public_transportation_081_0
1164
public_transportation_081_1
1165
public_transportation_083_0
1166
public_transportation_083_1
1167
public_transportation_111_0
1168
public_transportation_111_1
1169
public_transportation_111_2
1170
public_transportation_111_3
1171
public_transportation_145_0
1172
public_transportation_145_1
1173
public_transportation_145_2
1174
public_transportation_145_3
1175
public_transportation_207_0
1176
public_transportation_207_1
1177
public_transportation_207_2
1178
public_transportation_207_3
1179
public_transportation_296_0
1180
public_transportation_296_1
1181
public_transportation_296_2
1182
public_transportation_296_3
1183
public_transportation_296_4
1184
public_transportation_296_5
1185
public_transportation_296_6
1186
public_transportation_386_0
1187
public_transportation_386_1
1188
public_transportation_450_0
1189
public_transportation_450_1
1190
public_transportation_450_2
1191
public_transportation_450_3
1192
shoeshop_0